Working!!

In [1]:
import os
import numpy as np
import pandas as pd
import pathlib
import re
import PyPDF2

In [2]:
def nameMarker(text, ahliDewan, fileName):
    listDialog = []

    # get pages
    pattern = 'DR.*?\d{4}\s\d{1,3}'
    page_odd = re.findall(pattern, text)

    pattern = '\d{1,3}\sDR.*?\d{4}'
    page_even = re.findall(pattern, text)

    for namaTarget in ahliDewan.Nama:
        text = re.sub(namaTarget, '#namaTarget', text)

    for seat in ahliDewan.Kawasan:
        pattern = '\[' + seat + '.*?\#namaTarget'
        matches = re.findall(pattern, text, flags=re.IGNORECASE)

        for match in matches:
            dialog = {
                'dialog': match,
                'session': fileName,
                'page_odd': page_odd,
                'page_even': page_even                
            }
            listDialog.append(dialog)

    return listDialog

In [3]:
# pathing
PARENT_PATH = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATA_PATH = os.path.join(PARENT_PATH, 'data')
PDF = os.path.join(DATA_PATH, 'PDF')
TEXT = os.path.join(DATA_PATH, 'TEXT')
COMPILE_TEXT = os.path.join(TEXT, 'compile.txt')
DIALOG = os.path.join(TEXT, 'compileDialog.txt')
AHLI_DEWAN2 = os.path.join(DATA_PATH, 'ahliDewan2.csv')

TEST = os.path.join(PDF, 'hansard_14-03-k01-01.pdf')

In [4]:
ahliDewan2 = pd.read_csv(AHLI_DEWAN2)

In [5]:
listDialog = pd.DataFrame()

for head, body, files in os.walk(PDF):
    for file in files:
        with open(TEST, 'rb') as reader:
                # create reader variable that will read the pdffileobj
                pdfreader=PyPDF2.PdfFileReader(reader, strict=False)

                # get total pages
                totalPages = pdfreader.numPages

                for curPages in range(totalPages):
                    pageObj = pdfreader.getPage(curPages)
                    text = pageObj.extractText()
                    text = ' '.join(text.split())
                    
                    # marker
                    text = nameMarker(text, ahliDewan2, file)

                    # store in df
                    if text:
                        text = pd.DataFrame(text)
                        listDialog = pd.concat([listDialog, text], ignore_index=True)
        

In [9]:
listDialog

,dialog,session,page_odd,page_even
0,[Port Dickson]: [Bangun] Tuan Yang di -Pertua:...,hansard_14-03-k01-01.pdf,[DR.26.07.2021 3],[]
1,"[Semporna]: Tuan Yang di -Pertua, saya... Dato...",hansard_14-03-k01-01.pdf,[DR.26.07.2021 3],[]
2,[Semporna]: ...Peraturan 11. Tuan Yang di -Per...,hansard_14-03-k01-01.pdf,[DR.26.07.2021 3],[]
3,"[Bandar Kuching]: Tuan Yang di -Pertua, izinka...",hansard_14-03-k01-01.pdf,[DR.26.07.2021 3],[]
4,"[Bandar Kuching]: Okey, terima kasih. Tuan Yan...",hansard_14-03-k01-01.pdf,[DR.26.07.2021 3],[]
...,...,...,...,...
76001,"[Jelutong]: Tuan Yang di -Pertua, kenap a Tuan...",hansard_14-05-k01-01.pdf,[],[108 DR.26.07.2021]
76002,"[Jelutong]: No, Tuan Yang di -Pertua. Tuan Yan...",hansard_14-05-k01-01.pdf,[],[108 DR.26.07.2021]
76003,[Jelutong]: Why are you hijacking the House Tu...,hansard_14-05-k01-01.pdf,[],[108 DR.26.07.2021]
76004,[Jelutong]: Kami ingin tahu. You are hijacking...,hansard_14-05-k01-01.pdf,[DR.26.07.2021 109],[]
